In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') # 경고 문구 생략

# 데이터 경로
data_path = 'data/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 피처명 한글화

In [2]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가(루블)',
                                          'item_cnt_day': '일_판매량'})
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})
item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

### 데이터 다운캐스팅

In [3]:
sales_train

,날짜,월ID,상점ID,상품ID,판매가(루블),일_판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [4]:
def datatype(df):
    type_ = df.dtypes
    print(f'type:{type_}')
    print()

In [5]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = datatype(df)

type:날짜          object
월ID          int64
상점ID         int64
상품ID         int64
판매가(루블)    float64
일_판매량      float64
dtype: object

type:상점명     object
상점ID     int64
dtype: object

type:상품명       object
상품ID       int64
상품분류ID     int64
dtype: object

type:상품분류명     object
상품분류ID     int64
dtype: object

type:ID      int64
상점ID    int64
상품ID    int64
dtype: object



In [6]:
def datasize(df):
    size = df.memory_usage().sum() / 1024
    print(f'size:{size:.2f}KB')
    print()

In [7]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    datasize(df)

size:137618.05KB

size:1.06KB

size:519.73KB

size:1.44KB

size:5020.44KB



In [8]:
sales_train.memory_usage().sum() / 1024**2

134.3926239013672

In [9]:
def downcast(df):
    before_me = df.memory_usage().sum() / 1024
    for col in df.columns:
        dtype_name = df[col].dtype.name
        # 오브젝트 타입이면 패스
        if dtype_name == 'object':
            pass
        # bool 타입이면 int8
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int'):
            # to_numeric() -> 숫자로 바꿔줌
            # downcast -> 알맞게 사이즈 변경
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    after_me = df.memory_usage().sum() / 1024
    print('{:.2f}% 압축!'.format(100 * (before_me - after_me) / before_me))
    
    return df

In [10]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

50.00% 압축!
38.60% 압축!
54.15% 압축!
39.95% 압축!
70.83% 압축!


In [11]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    datasize(df)

size:68809.09KB

size:0.65KB

size:238.28KB

size:0.86KB

size:1464.38KB



### 데이터 조합 생성

In [12]:
sales_train['월ID'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
      dtype=int8)

In [13]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가(루블),일_판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [14]:
sales_train.loc[sales_train['월ID']==0, '상점ID'].unique()

array([59, 25, 24, 23, 19, 22, 18, 21, 28, 27, 29, 26,  4,  6,  2,  3,  7,
        0,  1, 16, 15,  8, 10, 14, 13, 12, 53, 31, 30, 32, 35, 56, 54, 47,
       50, 42, 43, 52, 51, 41, 38, 44, 37, 46, 45], dtype=int8)

In [15]:
sales_train.describe().T

,count,mean,std,min,25%,50%,75%,max
월ID,2935849.0,14.569911,9.422988,0.0,7.0,14.0,23.0,33.0
상점ID,2935849.0,33.001728,16.226973,0.0,22.0,31.0,47.0,59.0
상품ID,2935849.0,10197.227057,6324.297354,0.0,4476.0,9343.0,15684.0,22169.0
판매가(루블),2935849.0,890.853233,1729.799631,-1.0,249.0,399.0,999.0,307980.0
일_판매량,2935849.0,1.242641,2.612213,-22.0,1.0,1.0,1.0,2169.0


In [16]:
from itertools import product

# 월ID, 상점ID, 상품ID 프레임 생성 
train = []
for i in sales_train['월ID'].unique():
    about_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    about_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    # itertools의 product는 데카르트 곱으로 A집합의 x이면서 B집합에서는 y인 순서쌍들의 집합
    train.append(np.array(list(product([i], about_shop, about_item))))
# 피처 설정
features = ['월ID', '상점ID', '상품ID']
# 리스트 타입인 train을 DataFrame 타입으로 변환 
train = pd.DataFrame(np.vstack(train), columns=features)
train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


In [17]:
train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


### 타깃값(월간 판매량) 추가

In [18]:
# features 그룹화해 판매량 합 구하기 
target = sales_train.groupby(features).agg({'일_판매량': 'sum'})
target = target.reset_index()
target = target.rename(columns={'일_판매량': '월간 판매량'})

In [19]:
target

,월ID,상점ID,상품ID,월간 판매량
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1609119,33,59,22087,6.0
1609120,33,59,22088,2.0
1609121,33,59,22091,1.0
1609122,33,59,22100,1.0


In [20]:
# train과 target 병합
train = train.merge(target, on=features, how='left')

In [21]:
train

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


### 테스트 데이터 이어붙이기

In [22]:
test

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [23]:
# test['월ID'] = 34

In [24]:
test

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [25]:
test['월ID'] = 34
# 불필요한 ID 제거
total_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=features)

In [26]:
# 결측값 0 입력
total_data = total_data.fillna(0)

total_data

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


In [27]:
total_data

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


### 최종 데이터 생성

In [28]:
# 공통 항목을 기준으로 merge() 사용
total_data = total_data.merge(shops, on='상점ID', how='left')
total_data = total_data.merge(items, on='상품ID', how='left')
total_data = total_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
total_data = downcast(total_data)

16.33% 압축!


In [29]:
total_data.head()

,월ID,상점ID,상품ID,월간 판매량,상점명,상품명,상품분류ID,상품분류명
0,0,59,22154,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0.0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


In [30]:
total_data = total_data.drop(['상점명', '상품명', '상품분류명'], axis=1)

In [31]:
total_data

,월ID,상점ID,상품ID,월간 판매량,상품분류ID
0,0,59,22154,1.0,37
1,0,59,2552,0.0,58
2,0,59,2554,0.0,58
3,0,59,2555,0.0,56
4,0,59,2564,0.0,59
...,...,...,...,...,...
11128045,34,45,18454,0.0,55
11128046,34,45,16188,0.0,64
11128047,34,45,15757,0.0,55
11128048,34,45,19648,0.0,40


### 데이터 나누기

In [32]:
# 훈련 data -> 2013년 1월(0) 부터 2015년 9월(32)
X_train = total_data[total_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 data -> 가장 최신 데이터 2015년 10월(33)
X_valid = total_data[total_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 data -> 위에서 추가한 예측 데이터 == 34
X_test = total_data[total_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# target data
y_train = total_data[total_data['월ID'] < 33]['월간 판매량']
# 0~20 제한
y_train = y_train.clip(0, 20)
# target 검증 data
y_valid = total_data[total_data['월ID'] == 33]['월간 판매량']
y_valid = y_valid.clip(0, 20)

In [33]:
total_data[total_data['월ID'] < 33]['월간 판매량'].value_counts()

0.0      9100618
1.0      1037474
2.0       260432
3.0       101320
4.0        52368
          ...   
607.0          1
275.0          1
453.0          1
272.0          1
473.0          1
Name: 월간 판매량, Length: 441, dtype: int64

In [34]:
y_train.value_counts()

0.0     9101523
1.0     1037474
2.0      260432
3.0      101320
4.0       52368
5.0       31392
6.0       20390
7.0       13834
20.0      12231
8.0       10272
9.0        7402
10.0       5910
11.0       4418
12.0       3651
13.0       2997
14.0       2402
15.0       2026
16.0       1716
17.0       1556
18.0       1272
19.0       1092
Name: 월간 판매량, dtype: int64

In [35]:
# # 가비지 컬렉터
# import gc

# del target, shops, items, item_categories, total_data
# gc.collect();

### 모델 훈련 및 성능 검증

In [36]:
import lightgbm as lgb

In [37]:
# params = {
#     'boosting_type': 'gbdt',    # 부스팅 타입
#     'objective': 'regression',  # 손실 함수
#     'metric': 'mse',            # 평가 지표
#     'num_leaves': 31,           # 트리의 최대 잎(리프) 수
#     'learning_rate': 0.05,      # 학습률
#     'feature_fraction': 0.9,    # feature를 사용할 비율
#     'bagging_fraction': 0.8,    # 데이터 샘플링 비율
#     'bagging_freq': 5,          # 데이터 샘플링의 빈도
#     'max_depth': -1,            # 트리의 최대 깊이
#     'n_jobs': -1,               # CPU 사용 개수 (-1은 모든 CPU 사용)
#     'seed': 42                  # 랜덤 시드
# }

In [38]:
# Lightgbm 하이퍼파라미터 지정
params = {'metric': 'rmse', # 평가지표
          'num_leaves': 200, #  트리의 최대 잎(리프)수
          'learning_rate': 0.005, # 학습률
          'force_col_wise': True, # 열 단위 방식으로 데이터를 처리
          'random_state': 10} # 시드값 고정

# 카테고리 설정
cat_features = ['상점ID', '상품분류ID']

# Lightgbm 훈련 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
# Lightgbm 검증 데이터셋
dvalid = lgb.Dataset(X_valid, y_valid)

In [40]:
# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      early_stopping_rounds=200,
                      verbose_eval=50)  

[LightGBM] [Info] Total Bins 426
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 4
[LightGBM] [Info] Start training from score 0.299125
Training until validation scores don't improve for 200 rounds
[50]	training's rmse: 1.17964	valid_1's rmse: 1.09497
[100]	training's rmse: 1.15067	valid_1's rmse: 1.06904
[150]	training's rmse: 1.13134	valid_1's rmse: 1.05245
[200]	training's rmse: 1.11805	valid_1's rmse: 1.04128
[250]	training's rmse: 1.10846	valid_1's rmse: 1.03412
[300]	training's rmse: 1.10111	valid_1's rmse: 1.02937
[350]	training's rmse: 1.09545	valid_1's rmse: 1.02588
[400]	training's rmse: 1.09079	valid_1's rmse: 1.02325
[450]	training's rmse: 1.08645	valid_1's rmse: 1.021
[500]	training's rmse: 1.08255	valid_1's rmse: 1.01863
[550]	training's rmse: 1.07911	valid_1's rmse: 1.01693
[600]	training's rmse: 1.07551	valid_1's rmse: 1.01507
[650]	training's rmse: 1.07242	valid_1's rmse: 1.01367
[700]	training's rmse: 1.06922	valid_1's rmse

### 예측 및 결과 제출

In [48]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)
# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

In [42]:
len(preds)

214200

In [43]:
preds

array([0.61958461, 0.44393639, 0.42120161, ..., 0.1139789 , 0.08863488,
       0.08751282])

In [44]:
submission

,ID,item_cnt_month
0,0,0.619585
1,1,0.443936
2,2,0.421202
3,3,0.385909
4,4,0.552047
...,...,...
214195,214195,0.109116
214196,214196,0.108321
214197,214197,0.113979
214198,214198,0.088635


In [45]:
# del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
# gc.collect();

In [46]:
from sklearn.metrics import r2_score

In [47]:
y_test = total_data[total_data['월ID'] == 30]['월간 판매량']
y_test = y_test[:214200]
r2_score(y_test, preds)

-0.0702806385578627